# Association rule mining

In this notebook, you'll implement the basic pairwise association rule mining algorithm.

To keep the implementation simple, you will apply your implementation to a simplified dataset, namely, letters ("items") in words ("receipts" or "baskets"). Having finished that code, you will then apply that code to some grocery store market basket data. If you write the code well, it will not be difficult to reuse building blocks from the letter case in the basket data case.

In [1]:
import sys
print(f"=== Python version ===\n{sys.version}")

=== Python version ===
3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]


## Problem definition

Let's say you have a fragment of text in some language. You wish to know whether there are association rules among the letters that appear in a word. In this problem:

- Words are "receipts"
- Letters within a word are "items"

You want to know whether there are _association rules_ of the form, $a \implies b$, where $a$ and $b$ are letters. You will write code to do that by calculating for each rule its _confidence_, $\mathrm{conf}(a \implies b)$. "Confidence" will be another name for an estimate of the conditional probability of $b$ given $a$, or $\mathrm{Pr}[b \,|\, a]$.

## Sample text input

Let's carry out this analysis on a "dummy" text fragment, which graphic designers refer to as the [_lorem ipsum_](https://en.wikipedia.org/wiki/Lorem_ipsum):

In [2]:
latin_text = """
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, totam
rem aperiam eaque ipsa, quae ab illo inventore
veritatis et quasi architecto beatae vitae dicta
sunt, explicabo. Nemo enim ipsam voluptatem, quia
voluptas sit, aspernatur aut odit aut fugit, sed
quia consequuntur magni dolores eos, qui ratione
voluptatem sequi nesciunt, neque porro quisquam est,
qui dolorem ipsum, quia dolor sit amet consectetur
adipisci[ng] velit, sed quia non numquam [do] eius
modi tempora inci[di]dunt, ut labore et dolore
magnam aliquam quaerat voluptatem. Ut enim ad minima
veniam, quis nostrum exercitationem ullam corporis
suscipit laboriosam, nisi ut aliquid ex ea commodi
consequatur? Quis autem vel eum iure reprehenderit,
qui in ea voluptate velit esse, quam nihil molestiae
consequatur, vel illum, qui dolorem eum fugiat, quo
voluptas nulla pariatur?

At vero eos et accusamus et iusto odio dignissimos
ducimus, qui blanditiis praesentium voluptatum
deleniti atque corrupti, quos dolores et quas
molestias excepturi sint, obcaecati cupiditate non
provident, similique sunt in culpa, qui officia
deserunt mollitia animi, id est laborum et dolorum
fuga. Et harum quidem rerum facilis est et expedita
distinctio. Nam libero tempore, cum soluta nobis est
eligendi optio, cumque nihil impedit, quo minus id,
quod maxime placeat, facere possimus, omnis voluptas
assumenda est, omnis dolor repellendus. Temporibus
autem quibusdam et aut officiis debitis aut rerum
necessitatibus saepe eveniet, ut et voluptates
repudiandae sint et molestiae non recusandae. Itaque
earum rerum hic tenetur a sapiente delectus, ut aut
reiciendis voluptatibus maiores alias consequatur
aut perferendis doloribus asperiores repellat.
"""

print("First 100 characters:\n  {} ...".format(latin_text[:100]))

First 100 characters:
  
Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium,  ...


**Exercise 0** (ungraded). Look up and read the translation of _lorem ipsum_!

**Data cleaning.** Like most data in the real world, this dataset is noisy. It has both uppercase and lowercase letters, words have repeated letters, and there are all sorts of non-alphabetic characters. For our analysis, we should keep all the letters and spaces (so we can identify distinct words), but we should ignore case and ignore repetition within a word.

For example, the eighth word of this text is "error." As an _itemset_, it consists of the three unique letters, $\{e, o, r\}$. That is, treat the word as a set, meaning you only keep the unique letters.

This itemset has three possible _itempairs_: $\{e, o\}$, $\{e, r\}$, and $\{o, r\}$.

> Since sets are unordered, note that we would regard $\{e, o\} = \{o, e\}$, which is why we say there are only three itempairs, rather than six.

Start by writing some code to help "clean up" the input.

---

**Exercise 1** (2 points). Complete the following function, `normalize_string(s)`. The input `s` is a string (`str` object). The function should return a new string with (a) all characters converted to lowercase and (b) all non-alphabetic, non-whitespace characters removed.

> _Clarification_. Scanning the sample text, `latin_text`, you may see things that look like special cases. For instance, `inci[di]dunt` and `[do]`. For these, simply remove the non-alphabetic characters and only separate the words if there is explicit whitespace.
>
> For instance, `inci[di]dunt` would become `incididunt` (as a single word) and `[do]` would become `do` as a standalone word because the original string has whitespace on either side. A period or comma without whitespace would, similarly, just be treated as a non-alphabetic character inside a word _unless_ there is explicit whitespace. So `e pluribus.unum basium` would become `e pluribusunum basium` even though your common-sense understanding might separate `pluribus` and `unum`.
>
> _Hint_. Regard as a whitespace character anything "whitespace-like." That is, consider not just regular spaces, but also tabs, newlines, and perhaps others. To detect whitespaces easily, look for a "high-level" function that can help you do so rather than checking for literal space characters.

In [3]:
### Define demo inputs
demo_s_ex1 = latin_text[:100]
print(demo_s_ex1)


Sed ut perspiciatis, unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium, 


<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
sed ut perspiciatis unde omnis iste natus error sit
voluptatem accusantium doloremque laudantium
```
<!-- Include any shout outs here -->

In [4]:
import re

import string  

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [5]:
def normalize_string(s):
    assert type (s) is str
    cleaned_string = ""
    
    # Iterate through each character in the input string
    for char in s:
        # Check if the character is not in string.punctuation (i.e., it's not a punctuation character)
        if char not in string.punctuation:
            cleaned_string += char  # Add the character to the cleaned string
    
    return cleaned_string.lower()

In [6]:
normalize_string(latin_text)

'\nsed ut perspiciatis unde omnis iste natus error sit\nvoluptatem accusantium doloremque laudantium totam\nrem aperiam eaque ipsa quae ab illo inventore\nveritatis et quasi architecto beatae vitae dicta\nsunt explicabo nemo enim ipsam voluptatem quia\nvoluptas sit aspernatur aut odit aut fugit sed\nquia consequuntur magni dolores eos qui ratione\nvoluptatem sequi nesciunt neque porro quisquam est\nqui dolorem ipsum quia dolor sit amet consectetur\nadipiscing velit sed quia non numquam do eius\nmodi tempora incididunt ut labore et dolore\nmagnam aliquam quaerat voluptatem ut enim ad minima\nveniam quis nostrum exercitationem ullam corporis\nsuscipit laboriosam nisi ut aliquid ex ea commodi\nconsequatur quis autem vel eum iure reprehenderit\nqui in ea voluptate velit esse quam nihil molestiae\nconsequatur vel illum qui dolorem eum fugiat quo\nvoluptas nulla pariatur\n\nat vero eos et accusamus et iusto odio dignissimos\nducimus qui blanditiis praesentium voluptatum\ndeleniti atque corru

In [7]:
### test_cell_ex1
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1', 
    'func': normalize_string, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        's':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'str',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
print('Passed! Please submit.')

Passed! Please submit.


---

**Exercise 2** (1 point). Implement the following function, `get_normalized_words(s)`. It takes as input a string `s` (i.e., a `str` object). It should normalize `s` and then return a list of its words. (That is, the function should not assume that the input `s` is normalized yet.)

In [8]:
### Define demo inputs

demo_s_ex2 = latin_text[:33]
demo_s_ex2

'\nSed ut perspiciatis, unde omnis '

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
['sed', 'ut', 'perspiciatis', 'unde', 'omnis']
```
<!-- Include any shout outs here -->

In [9]:
def get_normalized_words (s):
    # normalize
    assert type (s) is str
    cleaned_string = ""
    
    # Iterate through each character in the input string
    for char in s:
        # Check if the character is not in string.punctuation (i.e., it's not a punctuation character)
        if char not in string.punctuation:
            cleaned_string += char  # Add the character to the cleaned string
    
    # word list
    words = cleaned_string.split()

    lowercase_words = [word.lower() for word in words]
    return lowercase_words

In [10]:
# Demo:
print(get_normalized_words(demo_s_ex2))

['sed', 'ut', 'perspiciatis', 'unde', 'omnis']


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 2. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [11]:
### test_cell_ex2

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_2', 
    'func': get_normalized_words, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        's':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


---

**Exercise 3** (2 points). Implement a function, `make_itemsets_unstructured_text(text)`. The input, `text`, is a string of unstructured text (like the `latin_text` example above). Your function should get the normalized words from the text, convert the characters of each word into an itemset and then return the list of all itemsets. These output itemsets should appear in the same order as their corresponding words in the input. You may find it helpful to call `get_normalized_words` in your solution.

In [12]:
### Define demo inputs
demo_text_ex3 = 'sed \tut, perspiciatis\n und.e omnis'

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
[{'d', 'e', 's'},
 {'t', 'u'},
 {'a', 'c', 'e', 'i', 'p', 'r', 's', 't'},
 {'d', 'e', 'n', 'u'},
 {'i', 'm', 'n', 'o', 's'}]
```
<!-- Include any shout outs here -->
> Because sets are unordered, different versions of Python may produce sets with whose element-ordering differs from what you see above. However, the sets themselves should be in this order in the output list, since that is the order in which the corresponding words were given.

In [13]:
def make_itemsets_unstructured_text(text):
    # normalize
    #assert type (s) is str
    cleaned_string = ""
    
    # Iterate through each character in the input string
    for char in text:
        # Check if the character is not in string.punctuation (i.e., it's not a punctuation character)
        if char not in string.punctuation:
            cleaned_string += char  # Add the character to the cleaned string
    
    # word list
    words = cleaned_string.split()

    lowercase_words = [word.lower() for word in words]
    
    # unique character set
    set_list = [{char for char in word} for word in lowercase_words]

    return set_list

In [14]:
make_itemsets_unstructured_text(demo_text_ex3)

[{'d', 'e', 's'},
 {'t', 'u'},
 {'a', 'c', 'e', 'i', 'p', 'r', 's', 't'},
 {'d', 'e', 'n', 'u'},
 {'i', 'm', 'n', 'o', 's'}]

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 3. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [15]:
### test_cell_ex3

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_3', 
    'func': make_itemsets_unstructured_text, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'text':{
            'dtype':'list', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'list',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


---

## Implementing the basic algorithm

Recall the pseudocode for the algorithm that Rachel and Rich derived together:

![FindAssocRules (pseudocode)](https://ndownloader.figshare.com/files/7635700?private_link=3c473609741895a5cc2c)

In the following series of exercises, let's implement this method. We'll build it "bottom-up," first defining small pieces and working our way toward the complete algorithm. This method allows us to test each piece before combining them.

Observe that the bulk of the work in this procedure is just updating these tables, $T$ and $C$. So your biggest implementation decision is how to store those. A good choice is to use a dictionary

---

## Aside: Default dictionaries

Recall that the overall algorithm requires maintaining a table of item-pair (tuples) counts. It would be convenient to use a dictionary to store this table, where keys refer to item-pairs and the values are the counts.

However, with Python's built-in dictionaries, you always to have to check whether a key exists before updating it. For example, consider this code fragment:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

D['existing-key'] += 1 # == 6
D['new-key'] += 1  # Error: 'new-key' does not exist!
```

The second attempt causes an error because `'new-key'` is not yet a member of the dictionary. So, a more correct approach would be to do the following:

```python
D = {'existing-key': 5} # Dictionary with one key-value pair

if 'existing-key' not in D:
    D['existing-key'] = 0
D['existing-key'] += 1
   
if 'new-key' not in D:
    D['new-key'] = 0
D['new-key'] += 1
```

This pattern is so common that there is a special form of dictionary, called a _default dictionary_, which is available from the `collections` module: [`collections.defaultdict`](https://docs.python.org/3/library/collections.html?highlight=defaultdict#collections.defaultdict).

When you create a default dictionary, you need to provide a "factory" function that the dictionary can use to create an initial value when the key does *not* exist. For instance, in the preceding example, when the key was not present the code creates a new key with the initial value of an integer zero (0). Indeed, this default value is the one you get when you call `int()` with no arguments:

In [16]:
print(int())

0


In [17]:
from collections import defaultdict

D2 = defaultdict(int) # Empty dictionary

D2['existing-key'] = 5 # Create one key-value pair

D2['existing-key'] += 1 # Update
D2['new-key'] += 1

print(D2)

defaultdict(<class 'int'>, {'existing-key': 6, 'new-key': 1})


---

**Exercise 4** (2 points). Start by implementing a function that enumerates all item-pairs within an itemset and updates, _in-place_, a table that tracks the counts of those item-pairs.

The signature of this function is:

```python
   def update_pair_counts(pair_counts, itemset):
       ...
```

where `pair_counts` is the table to update and `itemset` is the itemset from which you need to enumerate item-pairs. You may assume `pair_counts` is a default dictionary. Each key is a pair of items `(a, b)`, and each value is the count. You may assume all items in `itemset` are distinct, i.e., that you may treat it as you would any set-like collection. Since the function will modify `pair_counts`, it does not need to return an object.

In [18]:
### Define demo inputs
demo_itemset_ex4 = {'f', 'r', 'o', 'g'}
demo_pair_counts_ex4 = defaultdict(int)
demo_pair_counts_ex4.update(
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 1,
             ('r', 'o'): 1,
             ('e', 'r'): 1,
             ('r', 'e'): 1})

# This wrapper will return the updated pair_counts as a new object without modifying the original. 
# It's convenient for testing!
def update_pair_counts_wrapper(pair_counts, itemset):
    from copy import deepcopy
    pair_counts_cp = deepcopy(pair_counts)
    update_pair_counts(pair_counts_cp, itemset)
    return pair_counts_cp

In [19]:
pairs = [(x, y) for x in demo_itemset_ex4 for y in demo_itemset_ex4 if x != y]
pairs

[('r', 'g'),
 ('r', 'o'),
 ('r', 'f'),
 ('g', 'r'),
 ('g', 'o'),
 ('g', 'f'),
 ('o', 'r'),
 ('o', 'g'),
 ('o', 'f'),
 ('f', 'r'),
 ('f', 'g'),
 ('f', 'o')]

In [20]:
for pair in pairs:
    if pair not in demo_pair_counts_ex4:
        demo_pair_counts_ex4[pair]=0
    demo_pair_counts_ex4[pair]+=1

In [21]:
demo_pair_counts_ex4

defaultdict(int,
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 2,
             ('r', 'o'): 2,
             ('e', 'r'): 1,
             ('r', 'e'): 1,
             ('r', 'g'): 1,
             ('r', 'f'): 1,
             ('g', 'r'): 1,
             ('g', 'o'): 1,
             ('g', 'f'): 1,
             ('o', 'g'): 1,
             ('o', 'f'): 1,
             ('f', 'r'): 1,
             ('f', 'g'): 1,
             ('f', 'o'): 1})

In [22]:
from collections import defaultdict
import itertools # Hint!

def update_pair_counts (pair_counts, itemset):
    """
    Updates a dictionary of pair counts for
    all pairs of items in a given itemset.
    """
    assert type (pair_counts) is defaultdict

    pairs = [(x, y) for x in itemset for y in itemset if x != y]

    for pair in pairs:
        if pair not in pair_counts:
            pair_counts[pair]=0
        pair_counts[pair]+=1
    
    return pair_counts

In [23]:
update_pair_counts_wrapper(demo_pair_counts_ex4, demo_itemset_ex4)

defaultdict(int,
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 3,
             ('r', 'o'): 3,
             ('e', 'r'): 1,
             ('r', 'e'): 1,
             ('r', 'g'): 2,
             ('r', 'f'): 2,
             ('g', 'r'): 2,
             ('g', 'o'): 2,
             ('g', 'f'): 2,
             ('o', 'g'): 2,
             ('o', 'f'): 2,
             ('f', 'r'): 2,
             ('f', 'g'): 2,
             ('f', 'o'): 2})

---

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
defaultdict(int,
            {('o', 'e'): 1,
             ('e', 'o'): 1,
             ('o', 'r'): 2,
             ('r', 'o'): 2,
             ('e', 'r'): 1,
             ('r', 'e'): 1,
             ('f', 'o'): 1,
             ('o', 'f'): 1,
             ('f', 'r'): 1,
             ('r', 'f'): 1,
             ('f', 'g'): 1,
             ('g', 'f'): 1,
             ('o', 'g'): 1,
             ('g', 'o'): 1,
             ('r', 'g'): 1,
             ('g', 'r'): 1})
```
> Note: This displayed output is `demo_pair_counts_ex4` which was updated _in place_. Your solution does not need to return any object.

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 4. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [24]:
### test_cell_ex4

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_4', 
    'func': update_pair_counts_wrapper, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'pair_counts':{
            'dtype':'defaultdict', # data type of param.
            'check_modified':True,
        },
        'itemset':{
            'dtype':'set', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'defaultdict',
            'check_dtype': False,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise


print('Passed! Please submit.')

Passed! Please submit.


---

**Exercise 5** (2 points). Implement a procedure that, given an itemset, updates a table to track counts of each item.

As with the previous exercise, you may assume all items in the given itemset (`itemset`) are distinct, i.e., that you may treat it as you would any set-like collection. You may also assume the table (`item_counts`) is a default dictionary.

In [25]:
### Define demo inputs
demo_item_counts_ex5 = defaultdict(int)
demo_item_counts_ex5.update({'o': 1, 'e': 1, 'r': 1})
demo_itemset_ex5 = {'f', 'r', 'o', 'g'}

def update_item_counts_wrapper(item_counts, itemset):
    from copy import deepcopy
    item_counts_cp = deepcopy(item_counts)
    update_item_counts(item_counts_cp, itemset)
    return item_counts_cp

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
defaultdict(int, {'o': 2, 'e': 1, 'r': 2, 'f': 1, 'g': 1})
```
<!-- Include any shout outs here -->

In [26]:
for item in demo_itemset_ex5:
        if item not in demo_item_counts_ex5:
            demo_item_counts_ex5[item]=0
        demo_item_counts_ex5[item]+=1

In [27]:
demo_item_counts_ex5

defaultdict(int, {'o': 2, 'e': 1, 'r': 2, 'g': 1, 'f': 1})

In [28]:
def update_item_counts(item_counts, itemset):
    for item in itemset:
        if item not in item_counts:
            item_counts[item]=0
        item_counts[item]+=1
    

In [29]:
update_item_counts_wrapper(demo_item_counts_ex5, demo_itemset_ex5)

defaultdict(int, {'o': 3, 'e': 1, 'r': 3, 'g': 2, 'f': 2})

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 5. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [30]:
### test_cell_ex5

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_5', 
    'func': update_item_counts_wrapper, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'item_counts':{
            'dtype':'defaultdict', # data type of param.
            'check_modified':False,
        },
        'itemset':{
            'dtype':'set', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'defaultdict',
            'check_dtype': False,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


---

**Exercise 6** (2 points). Define `create_rules_from_counts` as follows: 
Given tables of item-pair counts (`pair_counts`) and individual item counts (`item_counts`) (You can assume both are default dictionaries), return all of the rules. The returned rules should be in the form of a dictionary whose key is the tuple, $(a, b)$ corresponding to the rule $a \Rightarrow b$, and whose value is the confidence of the rule, $\mathrm{conf}(a \Rightarrow b)$. 

You may assume that if $(a, b)$ is in the table of item-pair counts, then both $a$ and $b$ are in the table of individual item counts.

In [31]:
### Define demo inputs

demo_item_counts_ex7 = {'blue fish': 44, 'one fish': 47, 'red fish': 100, 'two fish': 74}
demo_pair_counts_ex7 = {('blue fish', 'one fish'): 16,
                        ('one fish', 'blue fish'): 16,
                        ('blue fish', 'red fish'): 36,
                        ('red fish', 'blue fish'): 36,
                        ('blue fish', 'two fish'): 27,
                        ('two fish', 'blue fish'): 27,
                        ('one fish', 'red fish'): 38,
                        ('red fish', 'one fish'): 38,
                        ('one fish', 'two fish'): 28,
                        ('two fish', 'one fish'): 28,
                        ('red fish', 'two fish'): 59,
                        ('two fish', 'red fish'): 59}

In [32]:
demo_item_counts_ex7  

{'blue fish': 44, 'one fish': 47, 'red fish': 100, 'two fish': 74}

In [33]:
demo_pair_counts_ex7

{('blue fish', 'one fish'): 16,
 ('one fish', 'blue fish'): 16,
 ('blue fish', 'red fish'): 36,
 ('red fish', 'blue fish'): 36,
 ('blue fish', 'two fish'): 27,
 ('two fish', 'blue fish'): 27,
 ('one fish', 'red fish'): 38,
 ('red fish', 'one fish'): 38,
 ('one fish', 'two fish'): 28,
 ('two fish', 'one fish'): 28,
 ('red fish', 'two fish'): 59,
 ('two fish', 'red fish'): 59}

In [34]:
first_values = list()

for key in demo_pair_counts_ex7.keys():
    first_values.append(key[0])

print(first_values)

['blue fish', 'one fish', 'blue fish', 'red fish', 'blue fish', 'two fish', 'one fish', 'red fish', 'one fish', 'two fish', 'red fish', 'two fish']


In [35]:
keys_list = list(demo_pair_counts_ex7.keys())
keys_list

[('blue fish', 'one fish'),
 ('one fish', 'blue fish'),
 ('blue fish', 'red fish'),
 ('red fish', 'blue fish'),
 ('blue fish', 'two fish'),
 ('two fish', 'blue fish'),
 ('one fish', 'red fish'),
 ('red fish', 'one fish'),
 ('one fish', 'two fish'),
 ('two fish', 'one fish'),
 ('red fish', 'two fish'),
 ('two fish', 'red fish')]

In [36]:
for pair in keys_list:
    first_fish = pair[0]
    first_fish_value = demo_item_counts_ex7[first_fish]
    confidence = demo_pair_counts_ex7[pair]/first_fish_value
    demo_pair_counts_ex7[pair] = confidence
    
demo_pair_counts_ex7

{('blue fish', 'one fish'): 0.36363636363636365,
 ('one fish', 'blue fish'): 0.3404255319148936,
 ('blue fish', 'red fish'): 0.8181818181818182,
 ('red fish', 'blue fish'): 0.36,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('two fish', 'blue fish'): 0.36486486486486486,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('red fish', 'one fish'): 0.38,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('two fish', 'one fish'): 0.3783783783783784,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}

In [37]:
def create_rules_from_counts(pair_counts, item_counts):
    rules = {} # (item_a, item_b) -> conf (item_a => item_b)
    keys_list = list(pair_counts.keys())
    
    for pair in keys_list:
        first_fish = pair[0]
        first_fish_value = item_counts[first_fish]
        confidence = pair_counts[pair]/first_fish_value
        rules[pair] = confidence

    return rules


In [38]:
rules = create_rules_from_counts(demo_pair_counts_ex7, demo_item_counts_ex7)
rules

{('blue fish', 'one fish'): 0.008264462809917356,
 ('one fish', 'blue fish'): 0.007243096423721141,
 ('blue fish', 'red fish'): 0.01859504132231405,
 ('red fish', 'blue fish'): 0.0036,
 ('blue fish', 'two fish'): 0.013946280991735538,
 ('two fish', 'blue fish'): 0.004930606281957633,
 ('one fish', 'red fish'): 0.017202354006337708,
 ('red fish', 'one fish'): 0.0038,
 ('one fish', 'two fish'): 0.012675418741511995,
 ('two fish', 'one fish'): 0.005113221329437546,
 ('red fish', 'two fish'): 0.0059,
 ('two fish', 'red fish'): 0.010774287801314827}

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('blue fish', 'one fish'): 0.36363636363636365,
 ('one fish', 'blue fish'): 0.3404255319148936,
 ('blue fish', 'red fish'): 0.8181818181818182,
 ('red fish', 'blue fish'): 0.36,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('two fish', 'blue fish'): 0.36486486486486486,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('red fish', 'one fish'): 0.38,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('two fish', 'one fish'): 0.3783783783783784,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}
```
<!-- Include any shout outs here -->
> Note: The items in the "counts" dictionaries are phrases, not just letters! You don't need any special logic to handle this. It's just something to notice.  
  
> Note: If you did the division indirectly your result might not match "exactly". There's more in other notebooks on why that is. However, we have accounted for it in the testing, so don't worry as long as you're "reasonably close".

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 6. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [39]:
### test_cell_ex6

from tester_fw.testers import Tester

conf = {
    'case_file':'tc_6', 
    'func': create_rules_from_counts, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'pair_counts':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'item_counts':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**Aside: pretty printing the rules.** The output of rules above is a little messy; here's a little helper function that structures that output a little, which will be useful for both debugging and reporting purposes.

In [40]:

def gen_rule_str(a, b, val=None, val_fmt='{:.3f}', sep=" = "):
    text = "{} => {}".format(a, b)
    if val:
        text = "conf(" + text + ")"
        text += sep + val_fmt.format(val)
    return text

def print_rules(rules):
    if type(rules) is dict or type(rules) is defaultdict:
        from operator import itemgetter
        ordered_rules = sorted(rules.items(), key=itemgetter(1), reverse=True)
    else: # Assume rules is iterable
        ordered_rules = [((a, b), None) for a, b in rules]
    for (a, b), conf_ab in ordered_rules:
        print(gen_rule_str(a, b, conf_ab))

# Demo:
print_rules(rules)

conf(blue fish => red fish) = 0.019
conf(one fish => red fish) = 0.017
conf(blue fish => two fish) = 0.014
conf(one fish => two fish) = 0.013
conf(two fish => red fish) = 0.011
conf(blue fish => one fish) = 0.008
conf(one fish => blue fish) = 0.007
conf(red fish => two fish) = 0.006
conf(two fish => one fish) = 0.005
conf(two fish => blue fish) = 0.005
conf(red fish => one fish) = 0.004
conf(red fish => blue fish) = 0.004


---

**Exercise 7** (1 Point). Given `rules`, a dictionary mapping pairs `(a, b)` to the confidence that `a` implies `b` as well as a `threshold`, define the function `filter_rules_by_conf`. It should return all the rules whose confidence is _at least_ the threshold.

In [41]:
### Define demo inputs

demo_rules_ex7 = {('blue fish', 'one fish'): 0.36363636363636365,
                    ('one fish', 'blue fish'): 0.3404255319148936,
                    ('blue fish', 'red fish'): 0.8181818181818182,
                    ('red fish', 'blue fish'): 0.36,
                    ('blue fish', 'two fish'): 0.6136363636363636,
                    ('two fish', 'blue fish'): 0.36486486486486486,
                    ('one fish', 'red fish'): 0.8085106382978723,
                    ('red fish', 'one fish'): 0.38,
                    ('one fish', 'two fish'): 0.5957446808510638,
                    ('two fish', 'one fish'): 0.3783783783783784,
                    ('red fish', 'two fish'): 0.59,
                    ('two fish', 'red fish'): 0.7972972972972973}
demo_threshold_ex7 = 0.59

In [42]:
keys_list = list(demo_rules_ex7.keys())

filtered = {}

for pair in keys_list:
    if demo_rules_ex7[pair]>demo_threshold_ex7:
        filtered[pair]=demo_rules_ex7[pair]
        
filtered

{('blue fish', 'red fish'): 0.8181818181818182,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('two fish', 'red fish'): 0.7972972972972973}

In [43]:
def filter_rules_by_conf(rules, threshold):
    
    keys_list = list(rules.keys())

    filtered = {}

    for pair in keys_list:
        if rules[pair]>=threshold:
            filtered[pair]=rules[pair]

    return filtered

In [44]:
filter_rules_by_conf(demo_rules_ex7, demo_threshold_ex7)

{('blue fish', 'red fish'): 0.8181818181818182,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('blue fish', 'red fish'): 0.8181818181818182,
 ('blue fish', 'two fish'): 0.6136363636363636,
 ('one fish', 'red fish'): 0.8085106382978723,
 ('one fish', 'two fish'): 0.5957446808510638,
 ('red fish', 'two fish'): 0.59,
 ('two fish', 'red fish'): 0.7972972972972973}
```
<!-- Include any shout outs here -->

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [45]:
### test_cell_ex7
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7', 
    'func': filter_rules_by_conf, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'rules':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 0
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


---

## Actual baskets!

Let's take a look at some real data that [someone](http://www.salemmarafi.com/code/market-basket-analysis-with-r/) was kind enough to prepare for a similar exercise designed for the R programming environment.

First, here's a code snippet to load the data, which is a text file. If you are running in the Vocareum environment, we've already placed a copy of the data there; if you are running outside, this code will try to download a copy from the CSE 6040 website.

Each line of this file is some customer's shopping basket. The items that the customer bought are stored as a comma-separated list of values.

In [46]:
def on_vocareum():
    import os
    return os.path.exists('.voc')

def download(file, local_dir="", url_base=None, checksum=None):
    import os, requests, hashlib, io
    local_file = "{}{}".format(local_dir, file)
    if not os.path.exists(local_file):
        if url_base is None:
            url_base = "https://cse6040.gatech.edu/datasets/"
        url = "{}{}".format(url_base, file)
        print("Downloading: {} ...".format(url))
        r = requests.get(url)
        with open(local_file, 'wb') as f:
            f.write(r.content)            
    if checksum is not None:
        with io.open(local_file, 'rb') as f:
            body = f.read()
            body_checksum = hashlib.md5(body).hexdigest()
            assert body_checksum == checksum, \
                "Downloaded file '{}' has incorrect checksum: '{}' instead of '{}'".format(local_file,
                                                                                           body_checksum,
                                                                                           checksum)
    print("'{}' is ready!".format(file))
    
if on_vocareum():
    DATA_PATH = "./resource/asnlib/publicdata/"
else:
    DATA_PATH = ""
datasets = {'groceries.csv': '0a3d21c692be5c8ce55c93e59543dcbe'}

for filename, checksum in datasets.items():
    download(filename, local_dir=DATA_PATH, checksum=checksum)

with open('{}{}'.format(DATA_PATH, 'groceries.csv')) as fp:
    groceries_file = fp.read()
print (groceries_file[0:250] + "...\n... (etc.) ...") # Prints the first 250 characters only
print("\n(All data appears to be ready.)")

'groceries.csv' is ready!
citrus fruit,semi-finished bread,margarine,ready soups
tropical fruit,yogurt,coffee
whole milk
pip fruit,yogurt,cream cheese ,meat spreads
other vegetables,whole milk,condensed milk,long life bakery product
whole milk,butter,yogurt,rice,abrasive clea...
... (etc.) ...

(All data appears to be ready.)


---

**Exercise 8** (3 Points). The groceries data is unfortunately in a different format than the letters we have been working with. We can process itemsets well enough with `update_pair_counts`, `update_item_counts`, `create_rules_from_counts`, and `filter_rules_by_conf`. Making the itemsets themselves is a different story. If we're going to work with this real data set then we have to make itemsets out of it. 

**Your task**: Complete the function `make_itemsets_csv`. Given `csv_str`, a string where each receipt is a _line_ and the items within each receipt are _separated by a comma_, return a list of sets. Each set should be a single receipt, and each element of the set should be an item contained within that receipt. As with the words the ordering within each itemset does not matter, however the order of the itemsets within the list should match the order in which they appear.

In [47]:
### Define demo inputs

demo_csv_str_ex8 = '''milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk'''
print(demo_csv_str_ex8)

milk,eggs,peanut butter,oatmeal
butter,pancake mix,maple syrup
dog treats,milk,milk


In [48]:
demo_csv_str_ex8

'milk,eggs,peanut butter,oatmeal\nbutter,pancake mix,maple syrup\ndog treats,milk,milk'

In [49]:
# Split the text into lines
lines = demo_csv_str_ex8.split('\n')

result = []

# Iterate through each line
for line in lines:
    # Split the line into items and convert them into a set
    itemset = set(line.split(','))
    
    # Append the set to the result list
    result.append(itemset)

print(result)

[{'oatmeal', 'milk', 'peanut butter', 'eggs'}, {'maple syrup', 'pancake mix', 'butter'}, {'milk', 'dog treats'}]


<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
[{'eggs', 'milk', 'oatmeal', 'peanut butter'},
 {'butter', 'maple syrup', 'pancake mix'},
 {'dog treats', 'milk'}]
```
<!-- Include any shout outs here -->

In [50]:
# ex8 solution
def make_itemsets_csv(csv_str):
    # Split the text into lines
    lines = csv_str.split('\n')

    result = []

    # Iterate through each line
    for line in lines:
        # Split the line into items and convert them into a set
        itemset = set(line.split(','))

        # Append the set to the result list
        result.append(itemset)
        
    return result

In [51]:
make_itemsets_csv(demo_csv_str_ex8)

[{'eggs', 'milk', 'oatmeal', 'peanut butter'},
 {'butter', 'maple syrup', 'pancake mix'},
 {'dog treats', 'milk'}]

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 8. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [52]:
### test_cell_ex8
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_8', 
    'func': make_itemsets_csv, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'csv_str':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**On higher order functions**  
Python functions are *objects*, just like strings, lists, dictionaries, etc. This means that they can be passed as arguments to other functions. (You have seen this before with sorting in Notebook 1.) Here's a short example, say I have this function.

```
def do_math_operation(x, y, func):
    return func(x, y)
```

Then I can define other functions and pass them to `do_math_operation` to change how it works. For instance let's say I have `add(a, b)` which returns the sum of `a` and `b`. I can call `do_math_operation(4, 5, add)` and the result will be the sum of `4+5`. I could also define a funciton called a function which multiplies two numbers pass that to change what `do_math_operation` does. For sure, that's not a very interesting example, but this is a very powerful concept. 

---

**Exercise 9** (4 Points). Use the tools you have created above to create a association rules from a data source. The following inputs:  
- `source`: the source data for your rules. You can assume this will be _either_ structured like the `latin_text` _or_ like the csv-formatted `groceries_file`. 
- `itemset_maker`: a function which can be used to transform the `source` into "itemsets". *Since you are given the parsing function as a parameter, you do not need to attempt to determine the format of `source`*. See "On higher order functions" above.
- `conf_threshold` and `min_count`: Your result should only include rules where `a` occurs in _at least_ `min_count` receipts _and_ $\mathrm{conf}(a \Rightarrow b)$ is at least `conf_threshold`

Return your confidence rules as a dictionary.
- Keys: pairs `(a,b)`
- Values: $\mathrm{conf}(a \Rightarrow b)$

> Note: We will test your solution using `make_itemsets_unstructured_text` and `make_itemsets_csv` on random inputs. These functions must be correctly defined above to pass the test cell. 

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750
conf(g => i) = 0.750
conf(f => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
conf(cocoa drinks => whole milk) = 0.591
conf(pudding powder => whole milk) = 0.565
conf(jam => whole milk) = 0.547
conf(cream => other vegetables) = 0.538
conf(cream => sausage) = 0.538
conf(baking powder => whole milk) = 0.523
conf(tidbits => rolls/buns) = 0.522
conf(rice => other vegetables) = 0.520
conf(cooking chocolate => whole milk) = 0.520
conf(specialty cheese => other vegetables) = 0.500
conf(rubbing alcohol => butter) = 0.500
conf(rubbing alcohol => citrus fruit) = 0.500
conf(ready soups => rolls/buns) = 0.500
conf(frozen fruits => whipped/sour cream) = 0.500
```
<!-- Include any shout outs here -->

> Note: The "Source: ...; Itemset Maker: ...; Confidence Threshold: ...; Min Count: ..." is just printed for convenience. It _should not be part of your output_!  
  
> Note: The rules are "pretty printed" using the `print_rules` function defined above. Your solution should output lists of dictionaries.  
  
> Note: The demo includes _two calls_ to your solution. The first is for `latin_rules`, and the second is for `groceries_file`.

In [53]:
itemset = make_itemsets_unstructured_text(latin_text)
itemset

[{'d', 'e', 's'},
 {'t', 'u'},
 {'a', 'c', 'e', 'i', 'p', 'r', 's', 't'},
 {'d', 'e', 'n', 'u'},
 {'i', 'm', 'n', 'o', 's'},
 {'e', 'i', 's', 't'},
 {'a', 'n', 's', 't', 'u'},
 {'e', 'o', 'r'},
 {'i', 's', 't'},
 {'a', 'e', 'l', 'm', 'o', 'p', 't', 'u', 'v'},
 {'a', 'c', 'i', 'm', 'n', 's', 't', 'u'},
 {'d', 'e', 'l', 'm', 'o', 'q', 'r', 'u'},
 {'a', 'd', 'i', 'l', 'm', 'n', 't', 'u'},
 {'a', 'm', 'o', 't'},
 {'e', 'm', 'r'},
 {'a', 'e', 'i', 'm', 'p', 'r'},
 {'a', 'e', 'q', 'u'},
 {'a', 'i', 'p', 's'},
 {'a', 'e', 'q', 'u'},
 {'a', 'b'},
 {'i', 'l', 'o'},
 {'e', 'i', 'n', 'o', 'r', 't', 'v'},
 {'a', 'e', 'i', 'r', 's', 't', 'v'},
 {'e', 't'},
 {'a', 'i', 'q', 's', 'u'},
 {'a', 'c', 'e', 'h', 'i', 'o', 'r', 't'},
 {'a', 'b', 'e', 't'},
 {'a', 'e', 'i', 't', 'v'},
 {'a', 'c', 'd', 'i', 't'},
 {'n', 's', 't', 'u'},
 {'a', 'b', 'c', 'e', 'i', 'l', 'o', 'p', 'x'},
 {'e', 'm', 'n', 'o'},
 {'e', 'i', 'm', 'n'},
 {'a', 'i', 'm', 'p', 's'},
 {'a', 'e', 'l', 'm', 'o', 'p', 't', 'u', 'v'},
 {'a'

In [54]:
pair_counts = defaultdict(int)

In [55]:
for i in itemset:
    update_pair_counts(pair_counts,i)

In [56]:
pair_counts

defaultdict(int,
            {('d', 's'): 20,
             ('d', 'e'): 27,
             ('s', 'd'): 20,
             ('s', 'e'): 45,
             ('e', 'd'): 27,
             ('e', 's'): 45,
             ('u', 't'): 57,
             ('t', 'u'): 57,
             ('t', 'e'): 70,
             ('t', 'p'): 30,
             ('t', 's'): 45,
             ('t', 'a'): 57,
             ('t', 'c'): 21,
             ('t', 'r'): 26,
             ('t', 'i'): 50,
             ('e', 't'): 70,
             ('e', 'p'): 28,
             ('e', 'a'): 52,
             ('e', 'c'): 20,
             ('e', 'r'): 48,
             ('e', 'i'): 47,
             ('p', 't'): 30,
             ('p', 'e'): 28,
             ('p', 's'): 21,
             ('p', 'a'): 30,
             ('p', 'c'): 10,
             ('p', 'r'): 19,
             ('p', 'i'): 26,
             ('s', 't'): 45,
             ('s', 'p'): 21,
             ('s', 'a'): 37,
             ('s', 'c'): 20,
             ('s', 'r'): 23,
             ('s', 'i'): 5

In [57]:
item_count = defaultdict(int)

for i in itemset:
    update_item_counts(item_count,i)

In [58]:
item_count

defaultdict(int,
            {'d': 50,
             's': 91,
             'e': 126,
             'u': 117,
             't': 114,
             'p': 45,
             'a': 108,
             'c': 37,
             'r': 60,
             'i': 120,
             'n': 65,
             'o': 81,
             'm': 72,
             'l': 55,
             'v': 22,
             'q': 39,
             'b': 16,
             'h': 6,
             'x': 6,
             'g': 8,
             'f': 8})

In [59]:
rules = create_rules_from_counts(pair_counts, item_count)

In [60]:
rules

{('d', 's'): 0.4,
 ('d', 'e'): 0.54,
 ('s', 'd'): 0.21978021978021978,
 ('s', 'e'): 0.4945054945054945,
 ('e', 'd'): 0.21428571428571427,
 ('e', 's'): 0.35714285714285715,
 ('u', 't'): 0.48717948717948717,
 ('t', 'u'): 0.5,
 ('t', 'e'): 0.6140350877192983,
 ('t', 'p'): 0.2631578947368421,
 ('t', 's'): 0.39473684210526316,
 ('t', 'a'): 0.5,
 ('t', 'c'): 0.18421052631578946,
 ('t', 'r'): 0.22807017543859648,
 ('t', 'i'): 0.43859649122807015,
 ('e', 't'): 0.5555555555555556,
 ('e', 'p'): 0.2222222222222222,
 ('e', 'a'): 0.4126984126984127,
 ('e', 'c'): 0.15873015873015872,
 ('e', 'r'): 0.38095238095238093,
 ('e', 'i'): 0.373015873015873,
 ('p', 't'): 0.6666666666666666,
 ('p', 'e'): 0.6222222222222222,
 ('p', 's'): 0.4666666666666667,
 ('p', 'a'): 0.6666666666666666,
 ('p', 'c'): 0.2222222222222222,
 ('p', 'r'): 0.4222222222222222,
 ('p', 'i'): 0.5777777777777777,
 ('s', 't'): 0.4945054945054945,
 ('s', 'p'): 0.23076923076923078,
 ('s', 'a'): 0.4065934065934066,
 ('s', 'c'): 0.21978021978

In [61]:
rules = filter_rules_by_conf(rules, 0.75)
rules

{('r', 'e'): 0.8,
 ('v', 't'): 0.8181818181818182,
 ('v', 'e'): 0.7727272727272727,
 ('q', 'u'): 1.0,
 ('h', 'i'): 0.8333333333333334,
 ('x', 'e'): 1.0,
 ('x', 'i'): 0.8333333333333334,
 ('b', 'i'): 0.75,
 ('g', 'i'): 0.75,
 ('f', 'i'): 0.75}

In [62]:
print_rules(rules)

conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750
conf(g => i) = 0.750
conf(f => i) = 0.750


---

---

In [63]:
itemset = make_itemsets_csv(groceries_file)
itemset

[{'citrus fruit', 'margarine', 'ready soups', 'semi-finished bread'},
 {'coffee', 'tropical fruit', 'yogurt'},
 {'whole milk'},
 {'cream cheese ', 'meat spreads', 'pip fruit', 'yogurt'},
 {'condensed milk',
  'long life bakery product',
  'other vegetables',
  'whole milk'},
 {'abrasive cleaner', 'butter', 'rice', 'whole milk', 'yogurt'},
 {'rolls/buns'},
 {'UHT-milk',
  'bottled beer',
  'liquor (appetizer)',
  'other vegetables',
  'rolls/buns'},
 {'pot plants'},
 {'cereals', 'whole milk'},
 {'bottled water',
  'chocolate',
  'other vegetables',
  'tropical fruit',
  'white bread'},
 {'bottled water',
  'butter',
  'citrus fruit',
  'curd',
  'dishes',
  'flour',
  'tropical fruit',
  'whole milk',
  'yogurt'},
 {'beef'},
 {'frankfurter', 'rolls/buns', 'soda'},
 {'chicken', 'tropical fruit'},
 {'butter', 'fruit/vegetable juice', 'newspapers', 'sugar'},
 {'fruit/vegetable juice'},
 {'packaged fruit/vegetables'},
 {'chocolate'},
 {'specialty bar'},
 {'other vegetables'},
 {'butter milk

Filter itemcount first before finding pair count. We need to filter itemcount based on the min_count threshold.

In [64]:
item_count = defaultdict(int)

for i in itemset:
    update_item_counts(item_count, i)

In [65]:
item_count

defaultdict(int,
            {'citrus fruit': 814,
             'ready soups': 18,
             'semi-finished bread': 174,
             'margarine': 576,
             'yogurt': 1372,
             'tropical fruit': 1032,
             'coffee': 571,
             'whole milk': 2513,
             'pip fruit': 744,
             'meat spreads': 42,
             'cream cheese ': 390,
             'condensed milk': 101,
             'long life bakery product': 368,
             'other vegetables': 1903,
             'rice': 75,
             'butter': 545,
             'abrasive cleaner': 35,
             'rolls/buns': 1809,
             'UHT-milk': 329,
             'bottled beer': 792,
             'liquor (appetizer)': 78,
             'pot plants': 170,
             'cereals': 56,
             'white bread': 414,
             'bottled water': 1087,
             'chocolate': 488,
             'curd': 524,
             'dishes': 173,
             'flour': 171,
             'beef': 516,
     

In [66]:
# Filter based on min_count

threshold = 10

item_count = {key: value for key, value in item_count.items() if value >= threshold}

In [67]:
item_count

{'citrus fruit': 814,
 'ready soups': 18,
 'semi-finished bread': 174,
 'margarine': 576,
 'yogurt': 1372,
 'tropical fruit': 1032,
 'coffee': 571,
 'whole milk': 2513,
 'pip fruit': 744,
 'meat spreads': 42,
 'cream cheese ': 390,
 'condensed milk': 101,
 'long life bakery product': 368,
 'other vegetables': 1903,
 'rice': 75,
 'butter': 545,
 'abrasive cleaner': 35,
 'rolls/buns': 1809,
 'UHT-milk': 329,
 'bottled beer': 792,
 'liquor (appetizer)': 78,
 'pot plants': 170,
 'cereals': 56,
 'white bread': 414,
 'bottled water': 1087,
 'chocolate': 488,
 'curd': 524,
 'dishes': 173,
 'flour': 171,
 'beef': 516,
 'frankfurter': 580,
 'soda': 1715,
 'chicken': 422,
 'fruit/vegetable juice': 711,
 'newspapers': 785,
 'sugar': 333,
 'packaged fruit/vegetables': 128,
 'specialty bar': 269,
 'butter milk': 275,
 'pastry': 875,
 'detergent': 189,
 'processed cheese': 163,
 'frozen dessert': 106,
 'root vegetables': 1072,
 'waffles': 378,
 'sweet spreads': 89,
 'salty snack': 372,
 'bathroom cl

The above item_count dictionary has been filtered based on the min_count threshold

In [68]:
pair_counts = defaultdict(int)

In [69]:
for i in itemset:
    update_pair_counts(pair_counts,i)

In [70]:
pair_counts

defaultdict(int,
            {('citrus fruit', 'ready soups'): 3,
             ('citrus fruit', 'semi-finished bread'): 24,
             ('citrus fruit', 'margarine'): 78,
             ('ready soups', 'citrus fruit'): 3,
             ('ready soups', 'semi-finished bread'): 4,
             ('ready soups', 'margarine'): 5,
             ('semi-finished bread', 'citrus fruit'): 24,
             ('semi-finished bread', 'ready soups'): 4,
             ('semi-finished bread', 'margarine'): 20,
             ('margarine', 'citrus fruit'): 78,
             ('margarine', 'ready soups'): 5,
             ('margarine', 'semi-finished bread'): 20,
             ('yogurt', 'tropical fruit'): 288,
             ('yogurt', 'coffee'): 96,
             ('tropical fruit', 'yogurt'): 288,
             ('tropical fruit', 'coffee'): 70,
             ('coffee', 'yogurt'): 96,
             ('coffee', 'tropical fruit'): 70,
             ('yogurt', 'pip fruit'): 177,
             ('yogurt', 'meat spreads'): 18,
   

I need to remove items that did not meet the min_count threshold from the itemset before after doing the pair_counts

In [71]:
pair_counts = {
    key: value for key, value in pair_counts.items()
    if all(item in item_count for item in key)
}

In [72]:
rules = create_rules_from_counts(pair_counts, item_count)

In [73]:
rules

{('citrus fruit', 'ready soups'): 0.0036855036855036856,
 ('citrus fruit', 'semi-finished bread'): 0.029484029484029485,
 ('citrus fruit', 'margarine'): 0.09582309582309582,
 ('ready soups', 'citrus fruit'): 0.16666666666666666,
 ('ready soups', 'semi-finished bread'): 0.2222222222222222,
 ('ready soups', 'margarine'): 0.2777777777777778,
 ('semi-finished bread', 'citrus fruit'): 0.13793103448275862,
 ('semi-finished bread', 'ready soups'): 0.022988505747126436,
 ('semi-finished bread', 'margarine'): 0.11494252873563218,
 ('margarine', 'citrus fruit'): 0.13541666666666666,
 ('margarine', 'ready soups'): 0.008680555555555556,
 ('margarine', 'semi-finished bread'): 0.034722222222222224,
 ('yogurt', 'tropical fruit'): 0.2099125364431487,
 ('yogurt', 'coffee'): 0.06997084548104957,
 ('tropical fruit', 'yogurt'): 0.27906976744186046,
 ('tropical fruit', 'coffee'): 0.06782945736434108,
 ('coffee', 'yogurt'): 0.1681260945709282,
 ('coffee', 'tropical fruit'): 0.12259194395796848,
 ('yogurt', 

In [74]:
rules = filter_rules_by_conf(rules, 0.5)
rules

{('rice', 'whole milk'): 0.6133333333333333,
 ('cereals', 'whole milk'): 0.6428571428571429,
 ('baking powder', 'whole milk'): 0.5229885057471264,
 ('specialty cheese', 'other vegetables'): 0.5,
 ('rice', 'other vegetables'): 0.52,
 ('honey', 'whole milk'): 0.7333333333333333,
 ('rubbing alcohol', 'butter'): 0.5,
 ('rubbing alcohol', 'whole milk'): 0.6,
 ('rubbing alcohol', 'citrus fruit'): 0.5,
 ('jam', 'whole milk'): 0.5471698113207547,
 ('ready soups', 'rolls/buns'): 0.5,
 ('cocoa drinks', 'whole milk'): 0.5909090909090909,
 ('cream', 'other vegetables'): 0.5384615384615384,
 ('tidbits', 'rolls/buns'): 0.5217391304347826,
 ('pudding powder', 'whole milk'): 0.5652173913043478,
 ('cream', 'sausage'): 0.5384615384615384,
 ('cooking chocolate', 'whole milk'): 0.52,
 ('frozen fruits', 'whipped/sour cream'): 0.5,
 ('frozen fruits', 'other vegetables'): 0.6666666666666666}

In [75]:
print_rules(rules)

conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
conf(cocoa drinks => whole milk) = 0.591
conf(pudding powder => whole milk) = 0.565
conf(jam => whole milk) = 0.547
conf(cream => other vegetables) = 0.538
conf(cream => sausage) = 0.538
conf(baking powder => whole milk) = 0.523
conf(tidbits => rolls/buns) = 0.522
conf(rice => other vegetables) = 0.520
conf(cooking chocolate => whole milk) = 0.520
conf(specialty cheese => other vegetables) = 0.500
conf(rubbing alcohol => butter) = 0.500
conf(rubbing alcohol => citrus fruit) = 0.500
conf(ready soups => rolls/buns) = 0.500
conf(frozen fruits => whipped/sour cream) = 0.500


---

---

In [80]:
def create_rules_from_source(source, itemset_maker, conf_threshold=0, min_count=0):
    
    itemset = itemset_maker(source)
    
    item_count = defaultdict(int)
    
    for i in itemset:
        update_item_counts(item_count,i)
        
    item_count = {key: value for key, value in item_count.items() if value >= min_count}
    
    pair_counts = defaultdict(int)
    
    for i in itemset:
        update_pair_counts(pair_counts, i)
        
    pair_counts = {
        key: value for key, value in pair_counts.items()
        if all(item in item_count for item in key)
    }
    
    rules = create_rules_from_counts(pair_counts, item_count)
    
    rules = filter_rules_by_conf(rules, conf_threshold)
    
    return rules

In [81]:
latin_rules = create_rules_from_source(latin_text, make_itemsets_unstructured_text, 0.75)
grocery_rules = create_rules_from_source(groceries_file, make_itemsets_csv, 0.5, 10)
print('Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0')
print_rules(latin_rules)
print()
print('Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10')
print_rules(grocery_rules)

Source: `latin_text`; Itemset Maker: `make_itemsets_unstructured_text`; Confidence Threshold: 0.75; Min Count: 0
conf(q => u) = 1.000
conf(x => e) = 1.000
conf(h => i) = 0.833
conf(x => i) = 0.833
conf(v => t) = 0.818
conf(r => e) = 0.800
conf(v => e) = 0.773
conf(b => i) = 0.750
conf(g => i) = 0.750
conf(f => i) = 0.750

Source: `groceries_file`; Itemset Maker: `make_itemsets_csv`; Confidence Threshold: 0.5; Min Count: 10
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(rice => whole milk) = 0.613
conf(rubbing alcohol => whole milk) = 0.600
conf(cocoa drinks => whole milk) = 0.591
conf(pudding powder => whole milk) = 0.565
conf(jam => whole milk) = 0.547
conf(cream => other vegetables) = 0.538
conf(cream => sausage) = 0.538
conf(baking powder => whole milk) = 0.523
conf(tidbits => rolls/buns) = 0.522
conf(rice => other vegetables) = 0.520
conf(cooking chocolate => whole milk) = 0.520
conf(specialty cheese => oth

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 9. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [82]:
### test_cell_ex9
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_9', 
    'func': create_rules_from_source, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'source':{
            'dtype':'string', # data type of param.
            'check_modified':False,
        },
        'itemset_maker':{
            'dtype':'function', # data type of param.
            'check_modified':False,
        },
        'conf_threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        },
        'min_count':{
            'dtype':'int', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


---

**Aside** In case you \*cough\* didn't get around to \*cough\* looking up and reading the translation of the Lorem Ipsum, here's the first two paragraphs. We're going to use it in the next exercise

In [84]:
english_text = """
But I must explain to you how all this mistaken idea
of denouncing of a pleasure and praising pain was
born and I will give you a complete account of the
system, and expound the actual teachings of the great
explorer of the truth, the master-builder of human
happiness. No one rejects, dislikes, or avoids
pleasure itself, because it is pleasure, but because
those who do not know how to pursue pleasure
rationally encounter consequences that are extremely
painful. Nor again is there anyone who loves or
pursues or desires to obtain pain of itself, because
it is pain, but occasionally circumstances occur in
which toil and pain can procure him some great
pleasure. To take a trivial example, which of us
ever undertakes laborious physical exercise, except
to obtain some advantage from it? But who has any
right to find fault with a man who chooses to enjoy
a pleasure that has no annoying consequences, or
one who avoids a pain that produces no resultant
pleasure?

On the other hand, we denounce with righteous
indignation and dislike men who are so beguiled and
demoralized by the charms of pleasure of the moment,
so blinded by desire, that they cannot foresee the
pain and trouble that are bound to ensue; and equal
blame belongs to those who fail in their duty
through weakness of will, which is the same as
saying through shrinking from toil and pain. These
cases are perfectly simple and easy to distinguish.
In a free hour, when our power of choice is
untrammeled and when nothing prevents our being
able to do what we like best, every pleasure is to
be welcomed and every pain avoided. But in certain
circumstances and owing to the claims of duty or
the obligations of business it will frequently
occur that pleasures have to be repudiated and
annoyances accepted. The wise man therefore always
holds in these matters to this principle of
selection: he rejects pleasures to secure other
greater pleasures, or else he endures pains to
avoid worse pains.
"""

**Exercise 10** (2 points). Let's consider the case when we have more than one "grocery list". We want to find the rules which are common to both. 

Given the following inputs complete the funtion `common_rules`:
- `source_0` and `source_1` - strings containing source data. These can be unstructured text, csv, or really anything that can be converted into "itemsets". You can assume that both are able to be processed by the same `itemset_maker`.
- `itemset_maker` - a function which will process a _single_ data source into itemsets.
- `conf_threshold` and `min_count`: Your result should only include rules where `a` occurs in _at least_ `min_count` receipts _and_ $\mathrm{conf}(a \Rightarrow b)$ is at least `conf_threshold`.

Your function should return a set of tuples, where each tuple is a key in both the rules generated from `source_0` and `source_1`.

> Note: We will test your solution using `make_itemsets_unstructured_text` and `make_itemsets_csv` on random inputs. These functions must be correctly defined above to pass the test cell. 

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{('q', 'u'), ('x', 'e')}
```
These are the keys for the rules common to both the Latin and English texts.

In [90]:
rule1 = create_rules_from_source(english_text, make_itemsets_unstructured_text, 0.75, 0)
rule1

{('x', 'e'): 1.0,
 ('k', 'e'): 0.7777777777777778,
 ('j', 'e'): 1.0,
 ('q', 'u'): 1.0,
 ('q', 'e'): 1.0,
 ('q', 'n'): 0.75,
 ('z', 'e'): 1.0,
 ('z', 'd'): 1.0,
 ('z', 'l'): 1.0,
 ('z', 'o'): 1.0,
 ('z', 'a'): 1.0,
 ('z', 'm'): 1.0,
 ('z', 'r'): 1.0,
 ('z', 'i'): 1.0}

In [91]:
rule2 = create_rules_from_source(latin_text, make_itemsets_unstructured_text, 0.75, 0)
rule2

{('r', 'e'): 0.8,
 ('v', 't'): 0.8181818181818182,
 ('v', 'e'): 0.7727272727272727,
 ('q', 'u'): 1.0,
 ('h', 'i'): 0.8333333333333334,
 ('x', 'e'): 1.0,
 ('x', 'i'): 0.8333333333333334,
 ('b', 'i'): 0.75,
 ('g', 'i'): 0.75,
 ('f', 'i'): 0.75}

In [93]:
common_keys = set(rule1.keys()) & set(rule2.keys())
common_keys

{('q', 'u'), ('x', 'e')}

In [94]:
# ex10 solution
def common_rules(source_0, source_1, itemset_maker, conf_threshold, min_count):
    rule0 = create_rules_from_source(source_0, itemset_maker, conf_threshold, min_count)
    rule1 = create_rules_from_source(source_1, itemset_maker, conf_threshold, min_count)
    common_keys = set(rule0.keys()) & set(rule1.keys())
    return common_keys


In [95]:
common_rules(latin_text, english_text, make_itemsets_unstructured_text, 0.75, 0)

{('q', 'u'), ('x', 'e')}

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 10. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [96]:
### test_cell_ex10
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_10', 
    'func': common_rules, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'source_0':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        },
        'source_1':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        },
        'itemset_maker':{
            'dtype':'function', # data type of param.
            'check_modified':False,
        },
        'conf_threshold':{
            'dtype':'float', # data type of param.
            'check_modified':False,
        },
        'min_count':{
            'dtype':'str', # data type of param.
            'check_modified':False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'set',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b's63L2lglDfBJpcKzxpwcyy61HyKnJNBOJXl9BMyWhyo=', path='resource/asnlib/publicdata/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

Passed! Please submit.


**Fin.** If you have made it this far, congratulations on completing the assignment. **Don't forget to submit!**